# UGP project

#### Author - Michał Dulski - 434678
#### Dataset - ag-news

## Installing Dependencies

In [2]:
%pip install 'transformers[torch]==4.34.1' tokenizers==0.14.1 sentencepiece==0.1.99 datasets==2.14.7 evaluate==0.4.1 sacrebleu==2.3.2 scikit-learn==1.3.1 protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalli

## Dataset preparation

In [3]:
#!/usr/bin/env python3

import json
import logging
from pathlib import Path

from datasets import load_dataset

LOGGER = logging.getLogger(__name__)

LABEL_MAP = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}

# ag-news train split size: 120k
# ag-news test split size: ~7.6k
# training split can be reduced to 5k but test split has to be splitted
# into ~3.8k validation and ~3.8k test therefore we'll reduce training
# dataset split to 5k and reduce test and validation splits to 3k

def reduce_and_save_train_dataset(file_path: Path, data_to_save: list[dict]) -> None:
    data_class_0, data_class_1, data_class_2, data_class_3 = [], [], [], []

    for data in data_to_save:
        label = data["label"]
        if (label == 0 or label == LABEL_MAP[0]):
            data_class_0.append(data)
        elif (label == 1 or label == LABEL_MAP[1]):
            data_class_1.append(data)
        elif (label == 2 or label == LABEL_MAP[2]):
            data_class_2.append(data)
        elif (label == 3 or label == LABEL_MAP[3]):
            data_class_3.append(data)

    quarter_of_dataset_size = 1250  # 5000 / 4

    reduced_dataset = data_class_0[:quarter_of_dataset_size] + data_class_1[:quarter_of_dataset_size] + data_class_2[:quarter_of_dataset_size] + data_class_3[:quarter_of_dataset_size]

    save_path = file_path.parent / f"{file_path.stem}-5k.json"
    with open(save_path, "wt") as f_write:
      for data_line in reduced_dataset:
        data_line_str = json.dumps(data_line)
        f_write.write(f"{data_line_str}\n")

def save_train_with_label_mapping(original_path: Path, data_to_save: list[dict]) -> None:
    file_name = "s2s-" + original_path.name
    file_path = original_path.parent / file_name
    LOGGER.info(data_to_save)

    with open(file_path, "wt") as f_write:
      for data_line in data_to_save:
        label = data_line["label"]
        translated_label = LABEL_MAP[label]
        data_line["label"] = translated_label
        data_line_str = json.dumps(data_line)
        f_write.write(f"{data_line_str}\n")

    LOGGER.info(data_to_save)
    reduce_and_save_train_dataset(file_path, data_to_save)

def reduce_and_save_valid_test_dataset(file_path: Path, data_to_save: list[dict]) -> None:
    data_class_0, data_class_1, data_class_2, data_class_3 = [], [], [], []

    for data in data_to_save:
        label = data["label"]
        if (label == 0 or label == LABEL_MAP[0]):
            data_class_0.append(data)
        elif (label == 1 or label == LABEL_MAP[1]):
            data_class_1.append(data)
        elif (label == 2 or label == LABEL_MAP[2]):
            data_class_2.append(data)
        elif (label == 3 or label == LABEL_MAP[3]):
            data_class_3.append(data)

    quarter_of_dataset_size = 750  # 3000 / 4

    reduced_dataset = data_class_0[:quarter_of_dataset_size] + data_class_1[:quarter_of_dataset_size] + data_class_2[:quarter_of_dataset_size] + data_class_3[:quarter_of_dataset_size]

    save_path = file_path.parent / f"{file_path.stem}-3k.json"
    with open(save_path, "wt") as f_write:
      for data_line in reduced_dataset:
        data_line_str = json.dumps(data_line)
        f_write.write(f"{data_line_str}\n")

def save_valid_test_with_label_mapping(original_path: Path, data_to_save: list[dict]) -> None:
    file_name = "s2s-" + original_path.name
    file_path = original_path.parent / file_name

    with open(file_path, "wt") as f_write:
      for data_line in data_to_save:
        label = data_line["label"]
        translated_label = LABEL_MAP[label]
        data_line["label"] = translated_label
        data_line_str = json.dumps(data_line)
        f_write.write(f"{data_line_str}\n")

    reduce_and_save_valid_test_dataset(file_path, data_to_save)


def main() -> None:
    news_dataset = load_dataset("ag_news")

    save_path = Path("data/")
    save_train_path = save_path / "train.json"
    save_valid_path = save_path / "valid.json"
    save_test_path = save_path / "test.json"

    if(not save_path.exists()):
      save_path.mkdir()


    data_train, data_valid, data_test = [], [], []

    for source_data, dataset in [
        (news_dataset["train"], data_train),
        (news_dataset["test"], data_valid),
    ]:
        for i, data in enumerate(source_data):
            data_line = {
                "label": int(data["label"]),
                "text": data["text"]
            }

            dataset.append(data_line)

    data_class_0, data_class_1, data_class_2, data_class_3 = [], [], [], []

    for data in data_valid:
        label = data["label"]
        if (label == 0):
            data_class_0.append(data)
        elif (label == 1):
            data_class_1.append(data)
        elif (label == 2):
            data_class_2.append(data)
        elif (label == 3):
            data_class_3.append(data)

    half_of_class_0_size = int(len(data_class_0) / 2)
    half_of_class_1_size = int(len(data_class_1) / 2)
    half_of_class_2_size = int(len(data_class_2) / 2)
    half_of_class_3_size = int(len(data_class_3) / 2)

    data_valid = data_class_0[:half_of_class_0_size] + data_class_1[:half_of_class_1_size] + data_class_2[:half_of_class_2_size] + data_class_3[:half_of_class_3_size]

    data_test = data_class_0[half_of_class_0_size:] + data_class_1[half_of_class_1_size:] + data_class_2[half_of_class_2_size:] + data_class_3[half_of_class_3_size:]

    for file_path, data_to_save in [
        (save_train_path, data_train),
        (save_valid_path, data_valid),
        (save_test_path, data_test)
    ]:
        with open(file_path, "wt") as f_write:
            for data_line in data_to_save:
                data_line_str = json.dumps(data_line)
                f_write.write(f"{data_line_str}\n")

        if file_path == save_train_path:
                reduce_and_save_train_dataset(file_path, data_to_save)
                save_train_with_label_mapping(file_path, data_to_save)
        else:
            reduce_and_save_valid_test_dataset(file_path, data_to_save)
            save_valid_test_with_label_mapping(file_path, data_to_save)

if __name__ == "__main__":
    main()

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

## GPU Info

In [4]:
!nvidia-smi

Sun Jan 14 19:36:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Running scripts


### run_glue.py

In [ ]:
def freeze_model_weights(model: torch.nn.Module) -> None:
    for param in model.parameters():
        param.requires_grad = False

gpt2_freeze_layers = [1, 3, 5]

class CustomGPT2ClassificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        hidden_size = config.n_embd
        self.dense_1 = nn.Linear(hidden_size, 2 * hidden_size)
        self.dense_2 = nn.Linear(2 * hidden_size, hidden_size)
        self.norm = nn.LayerNorm(2 * hidden_size)
        self.dropout = nn.Dropout(config.resid_pdrop)
        self.out_proj = nn.Linear(hidden_size, config.num_labels, bias=False)
        self.p_relu = nn.PReLU()

    def forward(self, x):
        x = self.dense_1(x)
        x = self.p_relu(x)
        x = self.norm(x)
        x = self.dropout(x)
        x = self.dense_2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class CustomGPT2ForSequenceClassification(GPT2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.transformer = GPT2Model(config)
        self.score = CustomGPT2ClassificationHead(config)

        for i, layer in enumerate(self.transformer.h):
            if i in gpt2_freeze_layers:
                freeze_model_weights(layer)

        # Model parallel
        self.model_parallel = False
        self.device_map = None

        # Initialize weights and apply final processing
        self.post_init()

## run_translation.py


In [ ]:
#!/usr/bin/env python
# coding=utf-8
# Copyright The HuggingFace Team and The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for sequence to sequence.
"""
# You can also adapt this script on your own sequence to sequence task. Pointers for this are left as comments.

import logging
import os
import sys
import warnings
from dataclasses import dataclass, field
from typing import Optional

import datasets
import evaluate
import numpy as np
import torch
from datasets import load_dataset

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    M2M100Tokenizer,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

from save_on_end_epoch import SaveOnEndEpochTrainerCallback

# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.34.0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/translation/requirements.txt")

logger = logging.getLogger(__name__)

# A list of all multilingual tokenizer which require src_lang and tgt_lang attributes.
MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast, M2M100Tokenizer]

LABEL_MAP = {
    "World": 0,
    "Sports": 1,
    "Business": 2,
    "Sci/Tech": 3
}

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    token: str = field(
        default=None,
        metadata={
            "help": (
                "The token to use as HTTP bearer authorization for remote files. If not specified, will use the token "
                "generated when running `huggingface-cli login` (stored in `~/.huggingface`)."
            )
        },
    )
    use_auth_token: bool = field(
        default=None,
        metadata={
            "help": "The `use_auth_token` argument is deprecated and will be removed in v4.34. Please use `token`."
        },
    )
    trust_remote_code: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether or not to allow for custom models defined on the Hub in their own modeling files. This option"
                "should only be set to `True` for repositories you trust and in which you have read the code, as it will"
                "execute code present on the Hub on your local machine."
            )
        },
    )
    freeze_weights: bool = field(
        default=False,
        metadata={"help": "Freeze encoder weights"},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    source_lang: str = field(default=None, metadata={"help": "Source language id for translation."})
    target_lang: str = field(default=None, metadata={"help": "Target language id for translation."})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a jsonlines)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the metrics (sacrebleu) on a jsonlines file."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to evaluate the metrics (sacrebleu) on a jsonlines file."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": (
                "The maximum total sequence length for target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "The maximum total sequence length for validation target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
                "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
                "during ``evaluate`` and ``predict``."
            )
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether to pad all samples to model maximum sentence length. "
                "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
                "efficient on GPU but very bad for TPU."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
                "which is used during ``evaluate`` and ``predict``."
            )
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default=None, metadata={"help": "A prefix to add before every source text (useful for T5 models)."}
    )
    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The token to force as the first generated token after the :obj:`decoder_start_token_id`.Useful for"
                " multilingual models like :doc:`mBART <../model_doc/mbart>` where the first generated token needs to"
                " be the target language token.(Usually it is the target language token)"
            )
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        elif self.source_lang is None or self.target_lang is None:
            raise ValueError("Need to specify the source language and the target language.")

        # accepting both json and jsonl file extensions, as
        # many jsonlines files actually have a .json extension
        valid_extensions = ["json", "jsonl"]

        if self.train_file is not None:
            extension = self.train_file.split(".")[-1]
            assert extension in valid_extensions, "`train_file` should be a jsonlines file."
        if self.validation_file is not None:
            extension = self.validation_file.split(".")[-1]
            assert extension in valid_extensions, "`validation_file` should be a jsonlines file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length


def freeze_model_weights(model: torch.nn.Module) -> None:
    for param in model.parameters():
        param.requires_grad = False


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if model_args.use_auth_token is not None:
        warnings.warn("The `use_auth_token` argument is deprecated and will be removed in v4.34.", FutureWarning)
        if model_args.token is not None:
            raise ValueError("`token` and `use_auth_token` are both specified. Please set only the argument `token`.")
        model_args.token = model_args.use_auth_token

    # Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
    # information sent is the one passed as arguments along with your Python/PyTorch versions.
    send_example_telemetry("run_translation", model_args, data_args)

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )

    if training_args.should_log:
        # The default of training_args.log_level is passive, so we set log level at info here to have that default.
        transformers.utils.logging.set_verbosity_info()

    log_level = training_args.get_process_log_level()
    logger.setLevel(log_level)
    datasets.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {training_args.parallel_mode.value == 'distributed'}, 16-bits training: {training_args.fp16}"
    )
    logger.info(f"Training/evaluation parameters {training_args}")

    if data_args.source_prefix is None and model_args.model_name_or_path in [
        "t5-small",
        "t5-base",
        "t5-large",
        "t5-3b",
        "t5-11b",
    ]:
        logger.warning(
            "You're running a t5 model but didn't provide a source prefix, which is expected, e.g. with "
            "`--source_prefix 'translate English to German: ' `"
        )

    # Detecting last checkpoint.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
        elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
            logger.info(
                f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own JSON training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For translation, only JSON files are supported, with one field named "translation" containing two keys for the
    # source and target languages (unless you adapt what follows).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            cache_dir=model_args.cache_dir,
            token=model_args.token,
        )
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
            extension = data_args.train_file.split(".")[-1]
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
            extension = data_args.validation_file.split(".")[-1]
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
            extension = data_args.test_file.split(".")[-1]
        raw_datasets = load_dataset(
            extension,
            data_files=data_files,
            cache_dir=model_args.cache_dir,
            token=model_args.token,
        )
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading.

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        token=model_args.token,
        trust_remote_code=model_args.trust_remote_code,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        token=model_args.token,
        trust_remote_code=model_args.trust_remote_code,
    )

    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        token=model_args.token,
        trust_remote_code=model_args.trust_remote_code,
    )

    # print model scheme
    print(model)

    # print frozen weights
    for name, param in model.named_parameters():
        if not param.requires_grad:
            print(name)

    if model_args.freeze_weights:
        logger.info("Freezing encoder weights")
        for i in range(1-3):
            freeze_model_weights(model.encoder.block[i])

    # We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
    # on a small vocab and want a smaller embedding size, remove this test.
    embedding_size = model.get_input_embeddings().weight.shape[0]
    if len(tokenizer) > embedding_size:
        model.resize_token_embeddings(len(tokenizer))

    # Set decoder_start_token_id
    if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
        if isinstance(tokenizer, MBartTokenizer):
            model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.target_lang]
        else:
            model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(data_args.target_lang)

    if model.config.decoder_start_token_id is None:
        raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

    prefix = data_args.source_prefix if data_args.source_prefix is not None else ""
    if 'classification' not in prefix:
        raise RuntimeError('Not found "classification" prefix!')
    prefix = prefix.strip()
    if not prefix.endswith(':'):
        prefix += ':'
    prefix += ' '
    logger.info(f'Using translation prefix: "{prefix!r}"')

    # Preprocessing the datasets.
    # We need to tokenize inputs and targets.
    if training_args.do_train:
        column_names = raw_datasets["train"].column_names
    elif training_args.do_eval:
        column_names = raw_datasets["validation"].column_names
    elif training_args.do_predict:
        column_names = raw_datasets["test"].column_names
    else:
        logger.info("There is nothing to do. Please pass `do_train`, `do_eval` and/or `do_predict`.")
        return

    # For translation we set the codes of our source and target languages (only useful for mBART, the others will
    # ignore those attributes).
    if isinstance(tokenizer, tuple(MULTILINGUAL_TOKENIZERS)):
        assert data_args.target_lang is not None and data_args.source_lang is not None, (
            f"{tokenizer.__class__.__name__} is a multilingual tokenizer which requires --source_lang and "
            "--target_lang arguments."
        )

        tokenizer.src_lang = "text"
        tokenizer.tgt_lang = "label"

        # For multilingual translation models like mBART-50 and M2M100 we need to force the target language token
        # as the first generated token. We ask the user to explicitly provide this as --forced_bos_token argument.
        forced_bos_token_id = (
            tokenizer.lang_code_to_id[data_args.forced_bos_token] if data_args.forced_bos_token is not None else None
        )
        model.config.forced_bos_token_id = forced_bos_token_id

    # Get the language codes for input/target.
    source_lang = data_args.source_lang.split("_")[0]
    target_lang = data_args.target_lang.split("_")[0]

    # Temporarily set max_target_length for training.
    max_target_length = data_args.max_target_length
    padding = "max_length" if data_args.pad_to_max_length else False

    if training_args.label_smoothing_factor > 0 and not hasattr(model, "prepare_decoder_input_ids_from_labels"):
        logger.warning(
            "label_smoothing is enabled but the `prepare_decoder_input_ids_from_labels` method is not defined for"
            f"`{model.__class__.__name__}`. This will lead to loss being calculated twice and will take up more memory"
        )

    def preprocess_function(examples):
        inputs = [ex for ex in examples[source_lang]]
        targets = [ex for ex in examples[target_lang]]
        inputs = [prefix + inp for inp in inputs]
        model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

        # Tokenize targets with the `text_target` keyword argument
        labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and data_args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    if training_args.do_train:
        if "train" not in raw_datasets:
            raise ValueError("--do_train requires a train dataset")
        train_dataset = raw_datasets["train"]
        if data_args.max_train_samples is not None:
            max_train_samples = min(len(train_dataset), data_args.max_train_samples)
            train_dataset = train_dataset.select(range(max_train_samples))
        with training_args.main_process_first(desc="train dataset map pre-processing"):
            train_dataset = train_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )

    if training_args.do_eval:
        max_target_length = data_args.val_max_target_length
        if "validation" not in raw_datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_dataset = raw_datasets["validation"]
        if data_args.max_eval_samples is not None:
            max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
            eval_dataset = eval_dataset.select(range(max_eval_samples))
        with training_args.main_process_first(desc="validation dataset map pre-processing"):
            eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on validation dataset",
            )

    if training_args.do_predict:
        max_target_length = data_args.val_max_target_length
        if "test" not in raw_datasets:
            raise ValueError("--do_predict requires a test dataset")
        predict_dataset = raw_datasets["test"]
        if data_args.max_predict_samples is not None:
            max_predict_samples = min(len(predict_dataset), data_args.max_predict_samples)
            predict_dataset = predict_dataset.select(range(max_predict_samples))
        with training_args.main_process_first(desc="prediction dataset map pre-processing"):
            predict_dataset = predict_dataset.map(
                preprocess_function,
                batched=True,
                num_proc=data_args.preprocessing_num_workers,
                remove_columns=column_names,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on prediction dataset",
            )

    # Data collator
    label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
    if data_args.pad_to_max_length:
        data_collator = default_data_collator
    else:
        data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id,
            pad_to_multiple_of=8 if training_args.fp16 else None,
        )

    # Metric
    metric = evaluate.load("sacrebleu")
    metric_accuracy = evaluate.load("accuracy")

    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        # Replace -100s used for padding as we can't decode them
        preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        decoded_preds_accuracy = [LABEL_MAP.get(decoded_pred, -1) for decoded_pred in decoded_preds]
        decoded_labels_accuracy = [LABEL_MAP.get(decoded_label[0], -1) for decoded_label in decoded_labels]

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result_accuracy = metric_accuracy.compute(predictions=decoded_preds_accuracy, references=decoded_labels_accuracy)
        result = {"bleu": result["score"], "accuracy": result_accuracy["accuracy"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

    # Initialize our Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if training_args.predict_with_generate else None,
        callbacks=[SaveOnEndEpochTrainerCallback()]
    )

    # Training
    if training_args.do_train:
        checkpoint = None
        if training_args.resume_from_checkpoint is not None:
            checkpoint = training_args.resume_from_checkpoint
        elif last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        trainer.save_model()  # Saves the tokenizer too for easy upload

        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Evaluation
    results = {}
    max_length = (
        training_args.generation_max_length
        if training_args.generation_max_length is not None
        else data_args.val_max_target_length
    )
    num_beams = data_args.num_beams if data_args.num_beams is not None else training_args.generation_num_beams
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        metrics = trainer.evaluate(max_length=max_length, num_beams=num_beams, metric_key_prefix="eval")
        max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    if training_args.do_predict:
        logger.info("*** Predict ***")

        predict_results = trainer.predict(
            predict_dataset, metric_key_prefix="predict", max_length=max_length, num_beams=num_beams
        )
        metrics = predict_results.metrics
        max_predict_samples = (
            data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
        )
        metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

        trainer.log_metrics("predict", metrics)
        trainer.save_metrics("predict", metrics)

        if trainer.is_world_process_zero():
            if training_args.predict_with_generate:
                predictions = predict_results.predictions
                predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
                predictions = tokenizer.batch_decode(
                    predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
                predictions = [pred.strip() for pred in predictions]
                output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
                with open(output_prediction_file, "w", encoding="utf-8") as writer:
                    writer.write("\n".join(predictions))

    kwargs = {"finetuned_from": model_args.model_name_or_path, "tasks": "translation"}
    if data_args.dataset_name is not None:
        kwargs["dataset_tags"] = data_args.dataset_name
        if data_args.dataset_config_name is not None:
            kwargs["dataset_args"] = data_args.dataset_config_name
            kwargs["dataset"] = f"{data_args.dataset_name} {data_args.dataset_config_name}"
        else:
            kwargs["dataset"] = data_args.dataset_name

    languages = [l for l in [data_args.source_lang, data_args.target_lang] if l is not None]
    if len(languages) > 0:
        kwargs["language"] = languages

    if training_args.push_to_hub:
        trainer.push_to_hub(**kwargs)
    else:
        trainer.create_model_card(**kwargs)

    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()


# Custom model and classification head can be found in custom_model.py file

# Encoder - RoBERTa
## Modifications

Model:
- custom classification head
    - danse layers
    - recurrent LSTM layer
    - multihead attention layer
    - normalization layer
    - ELU activation function
- frozen 2nd, 5th and 7th layers

Training arguments:
- --warmup_steps 60 # batch size 8, steps per epoch 5000/8=625, 625*10%=62,5, rounded down to 60
- --eval_steps 625 # evaluate model after each epoch

In [ ]:
roberta_freeze_layers = [1, 4, 6]
def freeze_model_weights(model: torch.nn.Module) -> None:
    for param in model.parameters():
        param.requires_grad = False

class CustomRobertaClassificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense1 = nn.Linear(config.hidden_size, config.hidden_size)
        self.lstm = nn.LSTM(config.hidden_size, config.hidden_size, batch_first=True)  # LSTM layer
        self.attention = nn.MultiheadAttention(config.hidden_size, num_heads=1)  # Attention layer
        self.dense2 = nn.Linear(config.hidden_size, config.hidden_size)
        self.norm = nn.LayerNorm(config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)
        self.elu = nn.ELU()

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dense1(x)
        x = self.elu(x)
        x, _ = self.lstm(x.unsqueeze(1))  # Pass through LSTM layer
        x = x.squeeze(1)
        x, _ = self.attention(x.unsqueeze(0), x.unsqueeze(0), x.unsqueeze(0))  # Pass through Attention layer
        x = x.squeeze(0)
        x = self.norm(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class CustomRobertaForSequenceClassification(RobertaForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.roberta = RobertaModel(config, add_pooling_layer=False)

        for i, layer in enumerate(self.roberta.encoder.layer):
            if i in roberta_freeze_layers:
                freeze_model_weights(layer)

        self.classifier = CustomRobertaClassificationHead(config)

        # Initialize weights and apply final processing
        self.post_init()

Model Scheme, frozen weights and whether the hidden state was used information is available in training history (run_glue.py lines 421, 441-446 )

### Execution

In [7]:
!python run_glue.py \
  --weight_decay 1e-5 \
  --warmup_steps 60 \
  --cache_dir .cache_training \
  --model_name_or_path roberta-base \
  --custom_model roberta_custom \
  --train_file data/train-5k.json  \
  --validation_file data/valid-3k.json \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --learning_rate 2e-5 \
  --num_train_epochs 1 \
  --save_strategy steps \
  --save_steps 625 \
  --save_total_limit 5 \
  --logging_strategy steps \
  --logging_steps 50 \
  --eval_steps 625 \
  --evaluation_strategy steps \
  --metric_for_best_model 'accuracy' \
  --greater_is_better 'True' \
  --load_best_model_at_end 'True' \
  --report_to 'none' \
  --output_dir out/ag-news/roberta_custom

2024-01-14 19:46:58.164416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 19:46:58.164469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 19:46:58.166060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 19:46:59.572013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
01/14/2024 19:47:02 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/14/2024 19:47:02 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adaf

# Decoder GPT2
## Modifications

Model:
- custom classification head
    - new dense layer
    - normalization layer
    - PReLU activation function
- frozen 2nd, 4th and 7th layers

Training arguments:
- --warmup_steps 60 # batch size 8, steps per epoch 5000/8=625, 625*10%=62,5, rounded down to 60
- --eval_steps 625 # evaluate model after each epoch

In [ ]:
def freeze_model_weights(model: torch.nn.Module) -> None:
    for param in model.parameters():
        param.requires_grad = False

gpt2_freeze_layers = [1, 3, 5]

class CustomGPT2ClassificationHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        hidden_size = config.n_embd
        self.dense_1 = nn.Linear(hidden_size, 2 * hidden_size)
        self.dense_2 = nn.Linear(2 * hidden_size, hidden_size)
        self.norm = nn.LayerNorm(2 * hidden_size)
        self.dropout = nn.Dropout(config.resid_pdrop)
        self.out_proj = nn.Linear(hidden_size, config.num_labels, bias=False)
        self.p_relu = nn.PReLU()

    def forward(self, x):
        x = self.dense_1(x)
        x = self.p_relu(x)
        x = self.norm(x)
        x = self.dropout(x)
        x = self.dense_2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class CustomGPT2ForSequenceClassification(GPT2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.transformer = GPT2Model(config)
        self.score = CustomGPT2ClassificationHead(config)

        for i, layer in enumerate(self.transformer.h):
            if i in gpt2_freeze_layers:
                freeze_model_weights(layer)

        # Model parallel
        self.model_parallel = False
        self.device_map = None

        # Initialize weights and apply final processing
        self.post_init()

Model Scheme, frozen weights and whether the hidden state was used information is available in training history (run_glue.py lines 421, 441-446 )

## Execution

In [9]:
!python run_glue.py \
  --weight_decay 1e-5 \
  --warmup_steps 60 \
  --cache_dir .cache_training \
  --model_name_or_path gpt2 \
  --custom_model gpt2_custom \
  --train_file data/train-5k.json  \
  --validation_file data/valid-3k.json \
  --per_device_train_batch_size 8 \
  --per_device_eval_batch_size 8 \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --learning_rate 2e-5 \
  --num_train_epochs 1 \
  --save_strategy steps \
  --save_steps 1000 \
  --save_total_limit 5 \
  --logging_strategy steps \
  --logging_steps 50 \
  --eval_steps 1000 \
  --evaluation_strategy steps \
  --metric_for_best_model 'accuracy' \
  --greater_is_better 'True' \
  --load_best_model_at_end 'True' \
  --report_to 'none' \
  --output_dir out/ag-news/gpt2_custom

2024-01-14 19:50:30.931012: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 19:50:30.931060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 19:50:30.932236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 19:50:32.033980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
01/14/2024 19:50:34 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/14/2024 19:50:34 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adaf

# Encoder-Decoder T5
## Modifications

Model:
- frozen layers 2-4

Training arguments:
- --warmup_steps 60 # batch size 8, steps per epoch 5000/8=625, 625*10%=62,5, round it down to 60
- --eval_steps 625 # evaluate model after each epoch
- --fp16 True

Model Scheme, frozen weights information is available in training history (run_translation.py lines 432-437)

Freezing the layers is done in run_translation.py file

In [11]:
!python run_translation.py \
  --fp16 True \
  --cache_dir .cache_training \
  --model_name_or_path "google/t5-v1_1-base" \
  --freeze_weights \
  --train_file data/s2s-train-5k.json \
  --validation_file data/s2s-valid-3k.json \
  --per_device_train_batch_size 32 \
  --per_device_eval_batch_size 32 \
  --source_lang "text" \
  --target_lang "label" \
  --source_prefix "ag-news classification" \
  --max_source_length 256 \
  --max_target_length 128 \
  --generation_max_length 128 \
  --do_train \
  --do_eval \
  --predict_with_generate \
  --num_train_epochs 3 \
  --save_strategy steps \
  --save_steps 1000 \
  --save_total_limit 5 \
  --logging_strategy steps \
  --logging_steps 50 \
  --eval_steps 1000 \
  --evaluation_strategy steps \
  --metric_for_best_model 'accuracy' \
  --greater_is_better 'True' \
  --load_best_model_at_end 'True' \
  --report_to 'none' \
  --output_dir out/ag-news/t5_v1_1-base

2024-01-14 20:00:55.825531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 20:00:55.825590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 20:00:55.827386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 20:00:58.851734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
01/14/2024 20:01:02 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
01/14/2024 20:01:02 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=

# Prompting FLAN-T5

In [12]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import torch

def get_pipeline(pipeline_type: str, model_name: str, model_type: str, torch_dtype: torch.dtype="auto", device_map="cuda"):
    if model_type == 's2s':
        class_type = AutoModelForSeq2SeqLM
    model = class_type.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch_dtype, device_map=device_map)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return pipeline(pipeline_type, model=model, tokenizer=tokenizer)

def generate_text_simple(model_pipeline, text: str, max_new_tokens: int = 20, is_prompt: bool = False):
    generated_text = model_pipeline(text, do_sample=False, max_new_tokens=max_new_tokens)[0]["generated_text"]
    if is_prompt and generated_text.startswith(text):
        generated_text = generated_text[len(text):].strip()
    return generated_text

lm_pipeline = get_pipeline('text2text-generation', 'google/flan-t5-base', 's2s', torch_dtype="auto", device_map="cpu")

labels = ["World", "Sports", "Business", "Sci/Tech"]

test_dataset = []
with open("data/s2s-test.json") as f:
    for line in f:
        test_dataset.append(json.loads(line))

correct = 0
total = 0

for item in test_dataset:
    text = item["text"]
    true_label = item["label"]

    prompt = f"{text} This text is about {labels[0]}, {labels[1]}, {labels[2]}, or {labels[3]}?"

    predicted_label = generate_text_simple(lm_pipeline, prompt)
    print(f"{prompt}\n{total}/3800: {predicted_label}")
    if predicted_label == true_label:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Total number of samples: {total}")
print(f"Correct predictions: {correct}")
print(f"Prompting accuracy: {accuracy}")

Streaming output truncated to the last 5000 lines.
1301/3800: Sports
Backman out as D-Backs manager The Diamondbacks will replace Wally Backman as manager, The Sporting News has confirmed, and his replacement will be Bob Melvin, according to the East Valley Tribune. This text is about World, Sports, Business, or Sci/Tech?
1302/3800: Sports
Diamondbacks Fire New Manager Backman Wally Backman is introduced as the new manager of the Arizona Diamondbacks during a news conference on Nov. 1, 2004, in Phoenix. Backman was fired Friday, Nov. 5, by the club. This text is about World, Sports, Business, or Sci/Tech?
1303/3800: Sports
Berkman tears ACL, may be out until June HOUSTON - Houston Astros star outfielder Lance Berkman suffered a torn ACL in his right knee and will undergo arthroscopic surgery within the next 10 days, the team announced Friday. This text is about World, Sports, Business, or Sci/Tech?
1304/3800: Sports
USC Releases Stewart From Scholarship (AP) AP - Southern California po

# Zero-shot FLAN-T5

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json

tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large')
model = AutoModelForSequenceClassification.from_pretrained('google/flan-t5-large').to('cuda')

labels = ["World", "Sports", "Business", "Sci/Tech"]

test_dataset = []
with open("data/s2s-test.json") as f:
    for line in f:
        test_dataset.append(json.loads(line))

correct = 0
total = 0

batch_size = 16

for i in range(0, len(test_dataset), batch_size):
    batch = test_dataset[i:i+batch_size]
    texts = [item["text"] for item in batch]
    true_labels = [item["label"] for item in batch]

    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    predicted_labels = [labels[i] for i in probabilities.argmax(dim=-1).cpu().numpy()]

    for j in range(len(batch)):
        print(f"{total}/3800\nInput: {texts[j]}\nPrediction {predicted_labels[j]}")

        if predicted_labels[j] == true_labels[j]:
            correct += 1
        total += 1

accuracy = correct / total
print(f"Total number of samples: {total}")
print(f"Correct predictions: {correct}")
print(f"Zero-shot learning accuracy: {accuracy}")

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-large and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Input: Consumer spending jumped in September WASHINGTON - Consumers, who substantially slowed down their spending in late summer, roared back to life in September, boosting their purchases by 0.6 percent.
Prediction World
2135/3800
Input: Sysco profit climbs 8 percent on sales Sysco Corp., the country #39;s largest food service distributor, Monday said profit for its latest quarter rose 8 percent, as it increased sales and trimmed expenses despite the hurricanes in the Southeast.
Prediction World
2136/3800
Input: Update 1: Sysco Profit Climbs 8 Percent on Sales Sysco Corp., the country #39;s largest food service distributor, Monday said profit for its latest quarter rose 8 percent, as it increased sales and trimmed expenses despite the hurricanes in the Southeast.
Prediction World
2137/3800
Input: Merck Shares Drop on Report of Documents About Vioxx Drug Merck  amp; Co. shares fell as much as 7 percent to their lowest level in more tha